### Import Libraries

In [1]:
import pandas as pd
key ='Your Key'
token = "Seller's Token"
devid = 'Your DevId'
certid = 'Your CertId'

from ebaysdk.trading import Connection as trading
from datetime import date
from datetime import timedelta

### Ebay GetSellingManagerSoldListings Call to get buyer's e-mail

In [2]:
# Define SaleDateRange
date_from   = "{}".format(date.today() - timedelta(90))
date_to     = "{}".format(date.today() - timedelta(minutes=120))

# Trading API Call to find out how many pages to load
api = trading(appid= key, devid=devid, certid=certid,  token=token, config_file=None)
response = api.execute("GetSellingManagerSoldListings",
    {
    "Archived" : "false",
    "SaleDateRange" : {
        "TimeFrom" : str(date_from),
        "TimeTo"   : str(date_to)
        },
    'Pagination':{'EntriesPerPage': '200'}
    }
)
# pages to load
pages = int(response.dict()['PaginationResult']['TotalNumberOfPages']) + 1

email_list = []
buyer_list = []
transaction_list = []
order_list = []

for page in range(1, pages):
    response = api.execute("GetSellingManagerSoldListings",
    {
    "Archived" : "false",
    "SaleDateRange" : {
        "TimeFrom" : str(date_from),
        "TimeTo"   : str(date_to)
        },
    'Pagination':{'PageNumber': str(page), 'EntriesPerPage': '200'}
    })
    for item in response.dict()["SaleRecord"]:
        buyer = item['BuyerID']
        buyer_list.append(buyer)
                    
        

        try:
            transaction_list.append(item['SellingManagerSoldTransaction']['TransactionID'])
        except TypeError:
            transaction = item['SellingManagerSoldTransaction'][0]['TransactionID']
            transaction_list.append(transaction)
        
        try:
            email = item['BuyerEmail']
            email_list.append(email)
        except KeyError:
            email_list.append(None)

        try:
            order = item['SellingManagerSoldTransaction']['OrderLineItemID']
            order_list.append(order)
        except:
            order_list.append(None)
            

### Create a DataFrame with results

In [6]:
data = pd.DataFrame({'buyer': buyer_list,
                     'e_mail': email_list,
                     'order_id':order_list,
                     'transaction_id': transaction_list})
print data.shape
data.head()

(776, 4)


,buyer,e_mail,order_id,transaction_id
0,shop4fun29,devory123@yahoo.com,252759953400-2170252681015,2170252681015
1,kop1tom,tvioux@hotmail.com,263308620913-2133787608016,2133787608016
2,nehchoks0,neha@vijisan.com,263282014002-2133759316016,2133759316016
3,gbailey*09,mbailey@gladstonellc.com,252967372142-2170049326015,2170049326015
4,sky_mms,murilomsantos@gmail.com,253122839968-2170007387015,2170007387015
